# 사용자 입력을 기다리는 방법 (Functional API)

!!! 정보 "전제 조건"
    이 가이드는 다음에 대한 지식이 있다고 가정합니다:

    - [휴먼 인 더 루프](../../concepts/human_in_the_loop) 워크플로우 구현 [인터럽트](../../concepts/human_in_the_loop/#interrupt)
    - [Functional API를 사용하여 ReAct 에이전트 생성하는 방법](../../how-tos/react-agent-from-scratch-functional)

**휴먼 인 더 루프 (HIL)** 상호작용은 [에이전틱 시스템](../../concepts/agentic_concepts/#human-in-the-loop)에서 중요합니다. 인간 입력을 기다리는 것은 일반적인 HIL 상호작용 패턴으로, 에이전트가 사용자에게 명확한 질문을 하고 입력을 기다린 다음 진행할 수 있도록 합니다.

우리는 LangGraph에서 [interrupt]() 함수를 사용하여 이를 구현할 수 있습니다. `interrupt`는 그래프 실행을 멈추고 사용자로부터 입력을 수집한 후 수집된 입력으로 실행을 계속할 수 있게 해줍니다.

이 가이드는 LangGraph의 [Functional API](../../concepts/functional_api)를 사용하여 휴먼 인 더 루프 워크플로우를 구현하는 방법을 보여줍니다. 구체적으로, 다음을 시연할 것입니다:

1. [간단한 사용 예시](#simple-usage)
2. [ReAct 에이전트와의 사용 방법](#agent)

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정합시다:


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
     <p class="admonition-title">더 나은 디버깅을 위해 <a href="https://smith.langchain.com">LangSmith</a>를 설정하세요</p>
     <p style="padding-top: 5px;">
         LangSmith에 가입하여 문제를 신속하게 파악하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버깅하고, 테스트하며 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">문서</a>를 참고하세요.
     </p>
</div>


## 간단한 사용법

간단한 사용 예제를 보여드리겠습니다. 우리는 세 가지 [작업](../../concepts/functional_api/#task)을 생성할 것입니다:

1. `"bar"`를 추가합니다.
2. 인간 입력을 위한 일시 중지. 재개할 때 인간 입력을 추가합니다.
3. `"qux"`를 추가합니다.


In [2]:
from langgraph.func import entrypoint, task
from langgraph.types import Command, interrupt


@task
def step_1(input_query):
    """Append bar."""
    return f"{input_query} bar"


@task
def human_feedback(input_query):
    """Append user input."""
    feedback = interrupt(f"Please provide feedback: {input_query}")
    return f"{input_query} {feedback}"


@task
def step_3(input_query):
    """Append qux."""
    return f"{input_query} qux"


이제 이러한 작업을 간단한 [엔트리포인트](../../concepts/functional_api/#entrypoint)에서 구성할 수 있습니다:


In [3]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def graph(input_query):
    result_1 = step_1(input_query).result()
    result_2 = human_feedback(result_1).result()
    result_3 = step_3(result_2).result()

    return result_3


우리가 인간-루프 워크플로우를 활성화하기 위해 수행한 모든 작업은 작업 내에서 [interrupt()](../../concepts/human_in_the_loop/#interrupt)이라고 불립니다.

!!! 팁

    이전 작업의 결과-- 이 경우 `step_1`--이 유지되므로 `interrupt` 이후에 다시 실행되지 않습니다.

쿼리 문자열을 보내봅시다:


In [4]:
config = {"configurable": {"thread_id": "1"}}


In [5]:
for event in graph.stream("foo", config):
    print(event)
    print("\n")


{'step_1': 'foo bar'}


{'__interrupt__': (Interrupt(value='Please provide feedback: foo bar', resumable=True, ns=['graph:d66b2e35-0ee3-d8d6-1a22-aec9d58f13b9', 'human_feedback:e0cd4ee2-b874-e1d2-8bc4-3f7ddc06bcc2'], when='during'),)}




`step_1` 이후에 `interrupt`로 일시 중지했음을 유의하십시오. 이 인터럽트는 실행을 재개하기 위한 지침을 제공합니다. 재개하려면 `human_feedback` 작업에서 기대되는 데이터가 포함된 [Command](../../concepts/human_in_the_loop/#the-command-primitive)를 발행합니다.


In [6]:
# Continue execution
for event in graph.stream(Command(resume="baz"), config):
    print(event)
    print("\n")


{'human_feedback': 'foo bar baz'}


{'step_3': 'foo bar baz qux'}


{'graph': 'foo bar baz qux'}




재개한 후에 실행은 남은 단계로 진행되며 예상대로 종료됩니다.


## 에이전트

[기능적 API를 사용하여 ReAct 에이전트를 생성하는 방법](../../how-tos/react-agent-from-scratch-functional) 가이드를 바탕으로 진행하겠습니다.

여기서는 필요할 때 인간에게 도움을 요청할 수 있도록 에이전트를 확장할 것입니다.

### 모델 및 도구 정의

예제를 위해 사용할 도구와 모델을 먼저 정의하겠습니다. [ReAct 에이전트 가이드](../../how-tos/react-agent-from-scratch-functional)와 마찬가지로, 특정 위치의 날씨에 대한 설명을 얻는 단일 자리 표시자 도구를 사용할 것입니다.

이 예제에서는 [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) 채팅 모델을 사용할 것이지만, 도구 호출을 지원하는 어떤 모델도 [사용 가능](https://python.langchain.com/docs/integrations/chat/)합니다.


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


도움을 받기 위해 사람에게 연락하려면, 단순히 [interrupt](../../concepts/human_in_the_loop/#interrupt)를 호출하는 도구를 추가하면 됩니다:


In [8]:
from langgraph.types import Command, interrupt


@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]


tools = [get_weather, human_assistance]


### 작업 정의

우리의 작업은 [ReAct 에이전트 가이드](../../how-tos/react-agent-from-scratch-functional)와 동일합니다:

1. **모델 호출**: 우리는 메시지 목록으로 우리의 채팅 모델에 쿼리를 하고자 합니다.
2. **도구 호출**: 만약 우리의 모델이 도구 호출을 생성하면, 우리는 그것을 실행하고자 합니다.

우리는 모델이 접근할 수 있는 또 하나의 도구만 더 있습니다.


In [9]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call)
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])


### 진입점 정의

우리의 [진입점](../../concepts/functional_api/#entrypoint)은 [ReAct 에이전트 가이드](../../how-tos/react-agent-from-scratch-functional)와 동일하게 유지됩니다:


In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages

checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):
    if previous is not None:
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(messages, [llm_response, *tool_results])

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return entrypoint.final(value=llm_response, save=messages)


### 사용법

모델을 인간의 도움이 필요한 질문으로 호출해 보겠습니다. 우리의 질문은 또한 `get_weather` 도구의 호출을 요구할 것입니다:


In [11]:
def _print_step(step: dict) -> None:
    for task_name, result in step.items():
        if task_name == "agent":
            continue  # just stream from tasks
        print(f"\n{task_name}:")
        if task_name == "__interrupt__":
            print(result)
        else:
            result.pretty_print()


In [12]:
config = {"configurable": {"thread_id": "1"}}


In [13]:
user_message = {
    "role": "user",
    "content": (
        "Can you reach out for human assistance: what should I feed my cat? "
        "Separately, can you check the weather in San Francisco?"
    ),
}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)


{'role': 'user', 'content': 'Can you reach out for human assistance: what should I feed my cat? Separately, can you check the weather in San Francisco?'}

call_model:
================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_joAEBVX7Abfm7TsZ0k95ZkVx)
 Call ID: call_joAEBVX7Abfm7TsZ0k95ZkVx
  Args:
    query: What should I feed my cat?
  get_weather (call_ut7zfHFCcms63BOZLrRHszGH)
 Call ID: call_ut7zfHFCcms63BOZLrRHszGH
  Args:
    location: San Francisco

call_tool:
================================= Tool Message =================================

content="It's sunny!" name='get_weather' tool_call_id='call_ut7zfHFCcms63BOZLrRHszGH'

__interrupt__:
(Interrupt(value={'query': 'What should I feed my cat?'}, resumable=True, ns=['agent:aa676ccc-b038-25e3-9c8a-18e81d4e1372', 'call_tool:059d53d2-3344-13bc-e170-48b632c2dd97'], when='during'),)


우리가 두 개의 도구 호출을 생성했음을 주의하세요. 우리의 실행이 중단되었지만, `get_weather` 도구의 실행은 차단하지 않았습니다.

우리가 중단된 곳을 살펴보겠습니다:


In [14]:
print(step)


{'__interrupt__': (Interrupt(value={'query': 'What should I feed my cat?'}, resumable=True, ns=['agent:aa676ccc-b038-25e3-9c8a-18e81d4e1372', 'call_tool:059d53d2-3344-13bc-e170-48b632c2dd97'], when='during'),)}


우리는 [명령](../../concepts/human_in_the_loop/#the-command-primitive)을 발행함으로써 실행을 재개할 수 있습니다. `Command`에서 제공하는 데이터는 `human_assistance`의 구현에 따라 귀하의 요구에 맞게 사용자 정의될 수 있음을 유의하세요.


In [15]:
human_response = "You should feed your cat a fish."
human_command = Command(resume={"data": human_response})

for step in agent.stream(human_command, config):
    _print_step(step)



call_tool:
================================= Tool Message =================================

content='You should feed your cat a fish.' name='human_assistance' tool_call_id='call_joAEBVX7Abfm7TsZ0k95ZkVx'

call_model:
================================== Ai Message ==================================

For human assistance, you should feed your cat fish. 

Regarding the weather in San Francisco, it's sunny!


위에서 재개할 때 최종 도구 메시지를 제공하여 모델이 응답을 생성할 수 있도록 합니다. 전체 실행 흐름을 보려면 LangSmith 추적을 확인하세요:

1. [초기 쿼리에서의 추적](https://smith.langchain.com/public/c3d8879d-4d01-41be-807e-6d9eed15df99/r)
2. [재개 후의 추적](https://smith.langchain.com/public/97c05ef9-8b4c-428e-8826-3fd417c8c75f/r)
